In [1]:
import pandas as pd
import openpyxl

## 文件读取和查看基本信息

In [2]:
# 设置 Excel 文件路径
path = 'OUTPUT91963302_5.xlsx'
# 读取 Excel 文件中的数据
df = pd.read_excel(path, engine='openpyxl')
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53709 entries, 0 to 53708
Data columns (total 38 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Times     53709 non-null  object
 1   t         53709 non-null  object
 2   STC(1)    53709 non-null  object
 3   STC(2)    53709 non-null  object
 4   STC(3)    53709 non-null  object
 5   STC(4)    53709 non-null  object
 6   STC(5)    53709 non-null  object
 7   STC(6)    53709 non-null  object
 8   STC(7)    53709 non-null  object
 9   STC(8)    53709 non-null  object
 10  STC(9)    53709 non-null  object
 11  STC(10)   53709 non-null  object
 12  STC(11)   53709 non-null  object
 13  STC(12)   53709 non-null  object
 14  STC(13)   53709 non-null  object
 15  STC(14)   53709 non-null  object
 16  STC(15)   53709 non-null  object
 17  STC(16)   53709 non-null  object
 18  STC(17)   53709 non-null  object
 19  STC(18)   53709 non-null  object
 20  SH2O(1)   53709 non-null  object
 21  SH2O(2)   53

## 数据筛选（根据年份）

In [3]:
df = df.iloc[2:]
print(df.shape)

(53707, 38)


In [4]:
# 筛选年份
bool_equation = df['Times']<20140101
print(bool_equation.shape)
df = df[bool_equation]
print(df.shape)

(53707,)
(35064, 38)


### 2010-2013 日平均

In [5]:
# 筛选STC，保留需要的STC列
sublist = [1,2,3,5,6,7,8]
# 根据列数生成列名字符串
subset = ['STC({})'.format(x) for x in sublist]
# 把时间列也加进去
subset.insert(0, 'Times')
print(subset)

['Times', 'STC(1)', 'STC(2)', 'STC(3)', 'STC(5)', 'STC(6)', 'STC(7)', 'STC(8)']


In [6]:
# 根据需要保留的STC列名列表提取数据
df_hour = df[subset]
df_hour.head()

,Times,STC(1),STC(2),STC(3),STC(5),STC(6),STC(7),STC(8)
2,20100101,252.464,254.969,257.693,263.165,264.828,268.266,272.763
3,20100101,254.654,255.594,257.577,263.073,264.824,268.261,272.75
4,20100101,258.061,257.209,257.948,262.982,264.819,268.256,272.737
5,20100101,261.983,259.485,258.798,262.901,264.811,268.251,272.768
6,20100101,265.203,261.799,259.946,262.839,264.801,268.246,272.755


In [7]:
# 获取天数列表
date = df_hour['Times'].drop_duplicates()
date

2        20100101
26       20100102
50       20100103
74       20100104
98       20100105
           ...   
34946    20131227
34970    20131228
34994    20131229
35018    20131230
35042    20131231
Name: Times, Length: 1461, dtype: object

In [9]:
# 创建一个新的DataFrame
result = pd.DataFrame(data=None, columns=subset[1:])
# 循环天数列表，计算每天的平均
for i in date:
#     print(i)
    # 提取一天24小时的STC
    _df = df_hour[df_hour['Times']==i]
    # 删除Times列
    _df = _df.iloc[:,1:]
    # 每列求平均
    _df = _df.mean(axis=0)
    # 将日均作为一行加入result中
    result.loc[i] = _df
    
# 温度单位转换  
result = result - 273.15
# 修改列名
result = result.reset_index(drop=False)
result.columns = subset
result = result.set_index('Times')
# 保存到xlsx(覆盖)
result.to_excel('output.xlsx', engine='openpyxl')
result.head()

,STC(1),STC(2),STC(3),STC(5),STC(6),STC(7),STC(8)
Times,,,,,,,
20100101,-12.582583,-12.205500,-11.741167,-9.909208,-8.338667,-4.940375,-0.396667
20100102,-12.144458,-11.808125,-11.446208,-9.818208,-8.307292,-5.043083,-0.431375
20100103,-10.098000,-10.225500,-10.309833,-9.550250,-8.235875,-5.125542,-0.483750
20100104,-8.525958,-8.750167,-8.952917,-8.850875,-7.973333,-5.175208,-0.517792
20100105,-11.754375,-11.017833,-10.206583,-8.446042,-7.605083,-5.157792,-0.564792
